# Determining the Effectiveness of the High Speed Test Track testing for the determination of accelerometer error coefficients. 
### By Sean Abrahamson 

This is a jupyter note book walking through the code for simulating an the performace of a single accelerometer output going down the Holloman High Speed Test Track and then using a the 746 TS Reference Position Vector to computer the error coefficients using least squares. 



### Import necessary libaries and functions from custom functions and classes from other jupiter notebooks.

In [3]:
import os.path
import pandas as pd
import numpy as np 
import plotly.graph_objects as go
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from classes_x import *
from scipy import integrate
from sigfig import round
import os
from sklearn.linear_model import LinearRegression

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'browser'

import pdb

# Import other Jupyter Notebooks
%run Thesis_Utils.ipynb

<a id='Accelerometer_Class'></a>

## Accelerometer Class

### Description:
The acceleromerter class defines an accelerometer object that contains an error model and a simulate function. The simulate function applies the associated error model attribute of the accelerombeter object and outputs what the output of the accelerometer would be given specific acceleration inputs. 
#### ___init___(self):
The accelerometer class is initiatlized with an error model that determines the kind of error the accelerometer demonstrates. For the purpose of this effort only the scale-factor non-linearity terms were added as they are a major focus of recent sled testing efforts. These values are estimated values for strategic grade resonating beam accelerometers. The values below can be found in the table below. 

| Coefficient	|Value	      |Units	       |Description                          |
| :---          | :---:       | :---:          | :---                                |
|$K_0$	        |5	          |$\mu g$	             |Bias                                 |
|$K_1$		    |.005         |$\mu g/g $	         |Scale Factor Error                   |
|$K_2$	        |60.144       |$\mu \frac{g}{g^2}$   |Scale factor 2nd order non-linearity |
|$K_3$	        |0.0152	      |$\mu \frac{g}{g^3}$   |Scale factor 3rd order non-linearity |
|$K_4$	        |0.0058	      |$\mu \frac{g}{g^4}$   |Scale factor 4th order non-linearity |
|$K_5$	        |0.0023       |$\mu \frac{g}{g^5}$   |Scale factor 5th order non-linearity |


#### function *simulate(self, a_i, n_start_idx, n_stop_idx)*

##### Description: 
The simulate function simulates the output of a single acceleromter given accelation $(A_i)$ in g's along it's input axis. The accelerometer error $(A_{err})$ is given by the below equation

$$A_{err}= K_0+K_1 A_{i}+K_2A_{i}^{2}+K_3A_{i}^{3}+K_4A_{i}^{4}+K_5A_{i}^{5}$$

To get the actual output of the acceleromter the computed error is converted to ($m/s^2$) then added to the original input acceleration

$$A_{sim} = g*(A_{err}) + A_{i}$$

In [5]:
class Accelerometer:
    
    def __init__(self):  # Default accelerometer characteristics
     
        self.g = 9.791807                     # Definition of g
        
        self.AccelModelCoef = {'K_1': 5            * 10**-6,      # Scale Factor (g/g) NEEDS UPDATED
                               'K_0': .005         * 10**-6,      # Bias (g)
                               'K_2': 61.14        * 10**-6,      # is second-order coefficient (g/g^2)
                               'K_3': 0.02         * 10**-6,      # is third-order coefficient  (g/g^3)
                               'K_4': 0.006        * 10**-6,      # is fourth-order coefficient (g/g^4)
                               'K_5': 0.0023       * 10**-6       # is fifth-order coefficient  (g/g^5)
                               }
        
        ## Other acceleromter error coefficients that could be added in the future.
        # self.K_0_asym = 0                   # Bias Asymmetry 
        # self.K_1_asym = 0                   # Scale Factor Asymmetry
        # self.K_oq = 0                       # Odd Quadratic Coefficient
        # self.omeg_o = 0                    # is misalignmet of the IA with respect to the OA
        # self.omeg_p = 0                    # is misalignmen of the IA with respect to the PA
        # self.K_ip = 0                      # is crosscoupling coefficient 
        # self.K_io = 0                      # is crosscoupling coefficient
        # self.K_po = 0                      # is crosscoupling coefficient
        # self.K_pp = 1.32E-4 * 10**-6       # is cross-axis nonlinearity coefficients
        # self.K_ppp = 2.10E-7 * 10**-6
        # self.K_pppp = 2.3E-10 * 10**-6
        # self.K_oo = 0                      # is cros-axis nonlinearity coefficients
        # self.K_spin = 0                    # is spin correction coefficient, equal to 
        # self.K_ang_accel = 0               # is angular acceleration coefficient
        
        
    def simulate(self,a_i,n_start_idx, n_stop_idx):
        """
        Starting with one dimensional error model. Outputs acceleration given
        true input acceleration. In the future errors caused by inputs along the pendulus axis (a_p) 
        and output axis (a_o) 
        could be added.
        """
        
        # Convert acceleration into g's since the error coefficients are defined in terms of g's. 
        g_i = a_i / self.g
        
        accel_model = [self.AccelModelCoef['K_1'] * (g_i),
                       self.AccelModelCoef['K_0'] * np.ones(len(g_i)),  
                       self.AccelModelCoef['K_2'] * (g_i**2), 
                       self.AccelModelCoef['K_3'] * (g_i**3), 
                       self.AccelModelCoef['K_4'] * (g_i**4), 
                       self.AccelModelCoef['K_5'] * (g_i**5)]
        
        # Add accelerometer error from each coefficient together and multiply by g then add original acceleration.
        a_x_Sim = self.g * sum(accel_model[n_start_idx:n_stop_idx]) + a_i
        
        
        return a_x_Sim
        

## Generate Reference Trajectory
### Description:
These set of functions are used to generate a sled test trajectory. 

##### importEGIData(Headers,filepath)
Imports data from a .csv file into columns titled using the inputed Headers list. 

In [6]:
def importEGIData(Headers,filepath):
        
    if filepath == '':
        print('No file selected')
    else: 
        D = pd.read_csv(filepath , names = Headers) # Pull only first row from Excel File

    return D

#### lpf(x, omege_c, T):
This function filters inputted data using a first order low pass filter. This is used to smooth out accelerometer data collected from a real sled test for use as data to create the reference trajectory.
 - x = Input array.
 - omega_c = Cutoff frequency.
 - T = Sample time. 

In [7]:
def lpf(x, omega_c, T):
    """Implement a first-order low-pass filter.
    
    The input data is x, the filter's cutoff frequency is omega_c 
    [rad/s] and the sample time is T [s].  The output is y.
    """
    N = np.size(x)
    y = x
    alpha = (2-T*omega_c)/(2+T*omega_c)
    beta = T*omega_c/(2+T*omega_c)
    for k in range(1, N):
        y[k] = alpha*y[k-1] + beta*(x[k]+x[k-1])
        
    return y 

#### generateReferenceTrajectory()
This function generates a reference trajectory given some inputted accleration data. For this implementation it takes in acceleration and velocity data from an EGI on board a sled test that took place at the HHSTT. 

generateReferenceTrajectory Steps:

Step 1: Import Data -
Real acceleration and velocity data was collected from an Embedded GPS/INS device mounted on a guidance sled test. All three axes of data is imported but only the values from the downtrack (X) axis is used in this implementation. 

Step 2: Cleaning data -
The data is then trimmed to focus on the part of the sled test where the actual launch occurs. The EGI sat for hours prior to launch during calibration of the unit under test but this data isn't necessary for this investigation. Occasionaly the time series from the raw data had repeated times for sucessive data points and so a new time series with an even sample rate was created and aplied to the data set to create a more even reference trajectory. The new time series for the data was created by taking the total duration of the trajetory and dividing it by the total number of data points in the trajectory to get the new sample rate.

Step 3: Smoothing Data - 
The data was then smoothed using a low pass filter to reduce high frequency noise and create a smoother trajectory to use as the reference trajectory.

Step 4: Save Data to DataFrame - 
Oranize the data by saving it to a single pandas dataframe.

Step 5: Create distance trajectories
The reference distance trajectory is made by double integrating the down track acceleration from the EGI. Motion prior to and after sled motion was forced to be zero where the actual measurement had some noise. The start time of the trajectory was also set to 0 meaning all data occuring prior to sled motion occured at "negative" time. 


The Pandas DataFrame with all the data stored is saved as a pickle file to be imported and used later.

In [8]:
def generateReferenceTrajectory(plotcheck = False):
    
    # Define the file paths for acceleration and velocity data. 
    Accel_filepath = './EGI_data/EGI_accel.csv'
    Vel_filepath = './EGI_data/EGI_accel.csv'
    
    # Save data into Pandas data frames with defined headers.
    EGI_accel = importEGIData(['Time', 'Ax','Ay','Az'],Accel_filepath)
    EGI_vel = importEGIData(['Time', 'Vx','Vy', 'Vz'],Vel_filepath)

    # Combine the Acceleration and Velocity data frames into one.
    EGI_accel_vel = EGI_accel.join(EGI_vel[['Vx','Vy','Vz']])

    ###############################################################################################################
    # Truth Gen Step 2 - Clean Data
    # Trim data to focus on actual sled run. These points were determined visually from the data used. 
    print('Developing Reference Trajectory')
    print("Trimming data to start/stop time determined visually...")
    startTime = 399600   # Index of data of the beginning of the reference trajectory
    stopTime = 399700    # Index of data of the end of the reference tracjectory

    # Trim the reference trajectory to the start and stop indicies defined above.  
    EGI_accel_vel_trim = EGI_accel_vel[(EGI_accel_vel['Time'] > startTime) & (EGI_accel_vel['Time'] < stopTime) ] # trim accelerometer output

    # The data used for creating the reference trajectory had repeated time values for multiple measurements of velocitty and acceleration.
    # To create a smooth reference trajectory the below code creates a new time series for the data by taking the total duration of the trajetory
    # and dividing it by the total number of data points in the trajectory so the new sample rate is even across the whole trajetory.
    
    # Determine new time series parameters
    # Tdur = total duration of data in seconds.
    # Tlen = number of data points
    Tdur = EGI_accel_vel_trim['Time'].max() - EGI_accel_vel_trim['Time'].min() 
    Tlen = len(EGI_accel_vel_trim['Time'])

    # Generate new time series given duration of trajectory and number of data points.
    NewTimeSeries = np.linspace(0, Tdur, Tlen)
    
    # Save new time series to Data Frame.
    EGI_accel_vel_trim.loc[:,'New Time'] = NewTimeSeries
     
    ###############################################################################################################    
    #%% Truth Gen Step 3 - Smooth Acceleration in X-axis
    # Pull data from data frame
    EGI_accel_presmoothed = EGI_accel_vel_trim[['Ax']]
    EGI_accel_smoothed_array = lpf(EGI_accel_vel_trim[['Ax']].to_numpy(),50,Tdur/Tlen)
    EGI_accel_vel_trim['Original_Ax'] = EGI_accel_presmoothed

    ###############################################################################################################
    #%% Truth Gen Step 4 - Create a DataFrame to house all truth data
    referenceTrajectory = pd.DataFrame()

    referenceTrajectory['Time'] = EGI_accel_vel_trim['New Time']
    referenceTrajectory['refAccel_x'] = EGI_accel_smoothed_array
    referenceTrajectory['refEGIVel_x'] = EGI_accel_vel_trim['Vx']

    ###############################################################################################################
    #%% Truth Gen Step 5 - Create distance trajectories.
    # Change initial acceleration in X to zero until launch. Determined visually
    print("Setting initial acceleration to 0 until launch...")
    referenceTrajectory['refAccel_x'][:1145] = 0

    # Change final acceleration after stop to zero. Determined visually
    print("Setting final acceleration at 0...")
    referenceTrajectory['refAccel_x'][4968:] = 0
    
    #%% Truth Gen Step 6 -  Integrate truth acceleration to get velocity and distance
    referenceTrajectory['refVel_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refAccel_x'],x = referenceTrajectory['Time'],initial = 0) 
    
    # Change final Velocity after stop to zero. Determined visually
    print("Setting final velocity at 0...")
    referenceTrajectory['refVel_x'][4968:] = 0
    
    # Integrate velocity to get trajectory distance. 
    referenceTrajectory['refDist_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refVel_x'],x = referenceTrajectory['Time'],initial = 0) 

    # Integrate EGI velocity to compare to double integrated acceleration
    referenceTrajectory['refEGIDist_x'] = integrate.cumulative_trapezoid(y = referenceTrajectory['refEGIVel_x'],x = referenceTrajectory['Time'],initial = 0) 
    
    # Compute start motion time.
    startMotionTime = referenceTrajectory['Time'][referenceTrajectory['refAccel_x']>0.001].iloc[0]
    # Set the start motion time as 0. 
    referenceTrajectory['Time'] = referenceTrajectory['Time']-startMotionTime
    
    
    #%% Save trajectory to Pickle File   
    referenceTrajectory.to_pickle("./referenceTrajectory.pkl")
    
    #%% Plots Acceleration and Velocity
    if plotcheck == True:
        Figure1 = PlotlyPlot()
        Figure1.setTitle('EGI Acceleration, Velocity and Smoothed acceleration')
        Figure1.setYaxisTitle('Acceleration (m/s/s)')
        Figure1.setYaxis2Title('Velocity (m/s)')
        Figure1.setXaxisTitle('GPS Time (s)')
        Figure1.settwoAxisChoice([False, True])
        Figure1.plotTwoAxis(referenceTrajectory[['Ax','Vx']], df_x= EGI_accel_vel_trim[['New Time']])
        Figure1.addLine(referenceTrajectory[['refAccel_x']], df_x = referenceTrajectory[['Time']],secondary_y=False)
        Figure1.show()
    return 



#### generateTrackRPV()

This function generates a reference position vector given a given reference trajector as created by generateReferenceTrajectory()


Step 1: Set up interupter system.  
The track reference is generated using interupter blades that are located approximately every 4.5 feet down the track. The system on the guidance sled records the time in which every interupter is passed and is then processed so that a downtrack distance of the sled is associated with every interupt.

The interupters are located at fixed downtrack distances so the reference trajectory can be used to determine at what time each interupter would be passed.

$$t=t_1+\left(I-R(t_1)\right) \frac{t_2-t_1}{R(t_2)-R(t_1)}$$

$R(t)$ = Reference Trajectory Down Track Distance at time $t$  
$T$ = Reference Tracjectory Time  
$I$ = Interupter Down Track Distance that falls between $R(t_2)$ and $R(t_1)$  
$t$ = Time

Step 2: Adding Error to Reference Position Vector  
This code has the ability to add a variety of errors to the RPV. Most notably the ability to add random noise given.

Noise is added using the following code (Zero mean, normal noise).  
    ```noise = np.random.normal(0,sigmaRPV,len(trackRPV)) # Add random noise to RPV```

In [9]:
def generateTrackRPV(referenceTrajectory, sigmaRPV, tauRPV, biasRPV, Overwrite=True):
    
    # print("\n Generating RPV...")
    trackRPV = pd.DataFrame()
        
    Interupter_delta = 4.5 * 0.3048 # ft converted to meters
    TrackLength = 10000   # Meters  
    
    trackRPV['Interupters_DwnTrk_dist'] = np.arange(0, TrackLength, Interupter_delta)
    trackRPV['Time'] = np.interp(trackRPV['Interupters_DwnTrk_dist'],referenceTrajectory['refDist_x'],referenceTrajectory['Time'])
    
    # Trim off the RPV so that there is no interupts past the time the sled stopped motion.
    trackRPV = trackRPV[trackRPV['Interupters_DwnTrk_dist'] <= referenceTrajectory['refDist_x'].max()]
    trackRPV = trackRPV.drop_duplicates(subset=['Time'])
    trackRPV = trackRPV[:-1]
    
    ###########################################################################################################
    #
    # REMOVED ZERO VELOCITY CODE. 
    # Code can be found in original python code. Remove
    # because adding areas of the trajectory where motion was
    # zero seemed to hurt estimates. 
    #
    ###########################################################################################################
    
    # Sort the values by time.
    trackRPV = trackRPV.sort_values(by='Time').reset_index(drop=True)
        
    # Add error to Track RPV
    if sigmaRPV != 0:
        noise = np.random.normal(0,sigmaRPV,len(trackRPV)) # Add random noise to RPV
        trackRPV['Interupters_DwnTrk_dist'] = trackRPV['Interupters_DwnTrk_dist'] + noise
    
    if tauRPV != 0: 
        trackRPV['Time'] = trackRPV['Time'] - tauRPV # Add time lag error
        
    if biasRPV != 0:
        trackRPV['Interupters_DwnTrk_dist'] = trackRPV['Interupters_DwnTrk_dist'] + biasRPV # Add distance bias.

    
    ###########################################################################################################
    #%% Save track RPV to pickle file
    if Overwrite == True:
        trackRPV.to_pickle(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")
    else:
       filepath = incrementFileName(f"./VarianceRPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")
       trackRPV.to_pickle(filepath)
    return

#### AccelSim()

ACCEL SIM - Scripts used to generate simulated accelerometer output based on truth input. Uses smoothed acceleration reference trajectory data to simulate. The simulated acceleration, velocity and distance is saved to a PandasData frame.

This method uses scipy's ```cumulative_trapezoid()``` to integrate acceleration to get velocity and distance. NOTE: This could potentially be improved with better methods.


See [Accelerometer Class](#Accelerometer_Class) for details on the acceleromter error model. 

In [10]:
def AccelSim(referenceTrajectory, N_model, changeDefaultCoeff, CoeffDict, g):
    
    #%% ACCEL SIM Step 1 - Simulate a Acceleromter with Bias using Accelerometer class
    """
    ACCEL SIM - Scripts used to generate simulated accelerometer output based on truth input
    
    Using smoothed acceleration truth data to simulate
    """
    
    # Intialize accelerometer class.
    AccelOne = Accelerometer()
    
    # Change the default error model coeficients defined in the Accleromter class. 
    if changeDefaultCoeff == True:
            AccelOne.AccelModelCoef.update(CoeffDict)
    
    # Create data frame to house data
    sensorSim = pd.DataFrame()
    sensorSim['Time'] = referenceTrajectory['Time']
    
    # Change to array for use in simulation.
    A_i_true = referenceTrajectory['refAccel_x'].to_numpy()  
    
    # Simulate
    A_x_sim = AccelOne.simulate(A_i_true, N_model[0], N_model[1])  
    
    # Store data in data frame. 
    sensorSim['SensorSim_Ax'] = A_x_sim
    
    # Accelerometer data is set to 0 prior to first motion. 
    sensorSim['SensorSim_Ax'][referenceTrajectory['refAccel_x'] == 0] = 0
    
    #%% Integrate Simulated accelerations to develop Velocity and Displacement.
    sensorSim['SensorSim_Vx'] = integrate.cumulative_trapezoid(y = sensorSim['SensorSim_Ax'],x = sensorSim['Time'],initial = 0) 
    sensorSim['SensorSim_Dx'] = integrate.cumulative_trapezoid(y = sensorSim['SensorSim_Vx'],x = sensorSim['Time'],initial = 0) 
    
    AccelObj = AccelOne
    
    return [sensorSim, AccelObj]

# RegressionAnalysis()

This function performs a least squares regression analysis on the simulated data to determine the error coefficients of the accelerometer.

The code you provided is performing a few mathematical operations. Here are the corresponding LaTeX equations:

1. Interpolation of Sensor Sim to Track:
   The code is interpolating the `SensorSim_Dx` values from `sensorSim` at the times specified in `trackRPV['Time']`. This can be represented by the following equation:

   $$SensorDwnTrkDist(t) = \text{interp}(t, sensorSim.Time, sensorSim.SensorSim\_Dx)$$

   where `interp` is the interpolation function, `t` is the time from `trackRPV['Time']`, and `sensorSim.Time` and `sensorSim.SensorSim_Dx` are the time and SensorSim_Dx columns from the sensorSim dataframe respectively.

2. Distance Error Calculation:
   The code is calculating the distance error as the difference between `Interupters_DwnTrk_dist` and `SensorDwnTrkDist`. This can be represented by the following equation:

   $$DistErr\_x = Interupters\_DwnTrk\_dist - SensorDwnTrkDist$$

3. Velocity Error Calculation:
   The code is calculating the velocity error as the difference in distance error over the difference in time. This can be represented by the following equation:

   $$Ve\_x = \frac{\Delta DistErr\_x}{\Delta t}$$

   where $\Delta DistErr\_x$ is the difference in distance error and $\Delta t$ is the difference in time.

4. Time Calculation for Velocity Error:
   The code is calculating the time associated with each velocity error as the average of two consecutive times. This can be represented by the following equation:

   $$Ve\_t = \frac{t_{i} + t_{i+1}}{2}$$

   where $t_{i}$ and $t_{i+1}$ are two consecutive times from `Dist_Error['Time']`.
   
   
  

5. If `sigmaRPV` is zero or `WLSoption` is false, then the size of matrix `W` is set to an identity matrix of the same size as matrix `A`. The covariance matrix `covariance_A` is set to 'Covariance Not Computed'.
6. Otherwise, the code computes the velocity error uncertainty `vel_sig` using the difference between consecutive elements of array `Ve_t`, which is stored in variable `delta_t`. The formula used for computing `vel_sig` is: 

$$ vel\_sig = \sqrt{2} \times \frac{\sigma_{RPV}}{\Delta t} $$

where $\sigma_{RPV}$ is a constant and $\Delta t$ is an array of time differences.

7. The code then computes the velocity error variance `vel_var` by squaring each element of array `vel_sig`.
8. The code then computes a weighted least squares weighting matrix `w` using the formula:

$$ w = diag(vel\_var,0) - diag(0.499 \times vel\_var[:-1],-1) - diag(0.499 \times vel\_var[:-1],1) $$

where `diag()` creates a diagonal matrix with the given input array as its diagonal elements.
9. The code then computes the inverse of matrix `w` and stores it in variable `W`.
10. The code checks if there are any negative values in the diagonal of matrix `W`. If there are any negative values, it prints a message indicating the total number of negative values detected.
11. Finally, the code saves matrix `w` as a CSV file named 'wMatrix' and computes the Cholesky decomposition of matrix `w`.


The difference between the reference position vector and the simulated accelerometer can be taken by first integrating the accelerometer values twice to get the simulated reported down track distance of the accelerometer. The  track reference position vector and simulated accelerometer distance is differences to get distance error. Then the distance error is differentiated with respect to time to get Velocity error. 

$$
D_e = D_{rpv} - D_{sim}
$$

$$
V_e = \frac{dD_e}{dt}
$$

Integrating the initial accelerometer error equation provides an equation relating the velocity error to the error model coefficients.   

$$
\overline{V_{e}}=V_0 + K_{0}t+K_{1} V_{X}+K_{2} \int A_{X}^{2}dt+K_{3}\int A_{X}^{3}dt+K_{4}\int A_{X}^{4}dt+K_{5}\int A_{X}^{5}dt
$$

The true velocity error is substituted with the error computer from the difference between the track RPV and simulated accelerometer. The equations can be rewritten as:

$$
V_e = Ax
$$

Where:

$$
A = \begin{bmatrix} 1 & t & V_x & \int A_{X}^{2}dt & \int A_{X}^{3}dt & \int A_{X}^{4}dt & \int A_{X}^{5}dt\end{bmatrix}
$$

$$
x = \begin{bmatrix} K_0 & K_1 & K_2 & K_3 & K_4 & K_5 \end{bmatrix}
$$

The coordinate functions (columns of A (reference IEEE)?) Are typically computed using the unit under test reported acceleration but since the true trajectory is know the coordinate functions are computed using the reference trajectory acceleration. For purposes of this study a simple cumulative trapezoid method was used to compute all integrations. 

To compute and estimate for the error model coefficients a least squares estimation is applied for all the values across the trajectory of the sled launch.


In [11]:
def RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g,sigmaRPV, startStopTime = [0, -1], saveToPickel = False, WLSoption = True, LeastSquaresMethod = 'LongHand',computeCovariance = True):

    """
    Error - Scripts used to compare accelerometer simulation versus track truth
    """
    Dist_Error = pd.DataFrame()
    Dist_Error['Time'] = trackRPV['Time']
    
    ##############################################################################################################
    # Step 1: Interpolate Sensor Sim to Track 
    ##############################################################################################################

    trackRPV['SensorDwnTrkDist'] = np.interp(trackRPV['Time'],sensorSim['Time'],sensorSim['SensorSim_Dx'])    
    Dist_Error['DistErr_x'] = trackRPV['Interupters_DwnTrk_dist'] - trackRPV['SensorDwnTrkDist']
    
    ##############################################################################################################
    # Step 2: Compute Velocity Error
    ##############################################################################################################
    # Compute Velocity Error
    Ve_x = (np.diff(Dist_Error['DistErr_x'])/np.diff(Dist_Error['Time']))
    Ve_t = (Dist_Error['Time'].head(-1) + np.diff(Dist_Error['Time'])/2).to_numpy()
    
   
    
    # Trim Ve_t and Ve_x to start and stop time.
    # first if startStopTime[1] == -1 then set stop time to end of data.
    if startStopTime[1] == -1:
        startStopTime = (startStopTime[0], Ve_t[-1])

    Ve_x = Ve_x[(Ve_t > startStopTime[0]) & (Ve_t <= startStopTime[1])]
    Ve_t = Ve_t[(Ve_t > startStopTime[0]) & (Ve_t <= startStopTime[1])]
    
    # Remove first two indicies as it causes problems with covariance matrix.
    idxStart = 10
    idxEnd = -10

    Ve_x = Ve_x[idxStart:idxEnd]
    Ve_t = Ve_t[idxStart:idxEnd]

    Error = pd.DataFrame()
    
    Error['Time'] = Ve_t
    Error['SensorSim_Ax'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Ax']) 
    Error['SensorSim_Vx'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Vx'])
    Error['SensorSim_Dx'] = np.interp(Ve_t,sensorSim['Time'],sensorSim['SensorSim_Dx'])
    Error['DistErr_x'] = np.interp(Ve_t,Dist_Error['Time'],Dist_Error['DistErr_x']) 
    Error['VelErr_x'] = Ve_x
     
    #%% - Regression Analysis
    """
    Regression Analysis - Scripts used to compute error model
    """
    
    
    ##############################################################################################################
    # Step X: Build A matrix based on accelerometer error model
    ##############################################################################################################
    
    # # Compute coordinate functions
    referenceTrajectory['Ax^2 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**2
    referenceTrajectory['Ax^3 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**3
    referenceTrajectory['Ax^4 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**4
    referenceTrajectory['Ax^5 (g)'] = (referenceTrajectory[['refAccel_x']]/g)**5
    
    referenceTrajectory['intAx^2 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^2 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    referenceTrajectory['intAx^3 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^3 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    referenceTrajectory['intAx^4 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^4 (g)'],x = referenceTrajectory['Time'],initial = 0)
    referenceTrajectory['intAx^5 (g)'] = -integrate.cumulative_trapezoid(y = referenceTrajectory['Ax^5 (g)'],x = referenceTrajectory['Time'],initial = 0) 
    
    
    Vx = np.interp(Ve_t, referenceTrajectory['Time'],referenceTrajectory['refVel_x'])
    intAx_2 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^2 (g)']) 
    intAx_3 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^3 (g)']) 
    intAx_4 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^4 (g)']) 
    intAx_5 = np.interp(Ve_t,referenceTrajectory['Time'],referenceTrajectory['intAx^5 (g)'])
    
    coordinateFunctionDF = pd.DataFrame()
    coordinateFunctionDF['Time'] = Ve_t
     
    coeff_dict = {'Est_V_0': 0, 'Est_K_1': 0, 'Est_K_0': 0, 'Est_K_2': 0, 'Est_K_3': 0, 'Est_K_4': 0, 'Est_K_5': 0}

    # Create Complete A Matrix
    complete_A = np.array([np.ones(len(Ve_t))/g, -Vx/g, -Ve_t, intAx_2, intAx_3, intAx_4, intAx_5])*g
    complete_A = complete_A.T
    
    complete_A_DF = pd.DataFrame(np.fliplr(complete_A), columns=['IntAx_5', 'IntAx_4', 'IntAx_3', 'IntAx_2', 'Ve_t', 'Vx', 'Ones'])
    
    trimmed_A_filt = np.zeros(complete_A.shape[1], dtype = bool)
    trimmed_A_filt[0] = 1
    
    trimmed_A_filt[N_model[0]+1:N_model[1]+1] = 1

    trimmed_A = complete_A[:,trimmed_A_filt]

    # Remove the first row of trimmed_A matrix
    A = trimmed_A[1:,:]    
    At = np.transpose(A)
    
    '''
    COMPUTE COVARIANCE
    '''
    
    ##############################################################################################################
    # Step X: Build the Weighting matrix and compute covariance matrix.
    ##############################################################################################################
    
    
    #%% 
    # Linear Regression
    coeff_list = tuple(None for _ in range(trimmed_A.shape[1]))

    if sigmaRPV == 0 or WLSoption == False: 
        # Remove first row of A to make consistent size with W
        size = A.shape[0]
        W = np.identity(size)
        covariance_A = 'Covariance Not Computed'
    
    else: 
        
        ################################################################################################
        # Develop weighted matrix
        ################################################################################################
        # Compute Velocity Error Uncertainty
        delta_t = np.diff(Ve_t)
        vel_sig = np.sqrt(2)*sigmaRPV/delta_t
        
        # Compute Velocity Error Variance
        vel_var = np.square(vel_sig)
        
        # Computed Weighted Least Squares Weighting Matrix
        w = np.diag(vel_var,0) - np.diag((.5*vel_var[:-1]),-1) - np.diag((.5*vel_var[:-1]),1)  
        W = np.linalg.inv(w)
        # W = np.diag(1/vel_sig) 

        ################################################################################################     
        ## TODO
        ## ADD Noise to velocity then add weighted velocity. 
        ## Try covariance on just individual coefficients. 
        ## Figure out if covariances are always off by certain amount.
        ################################################################################################

        # pdb.set_trace()    
            
        negative_values = np.sum(np.diag(W) < 0)
        if negative_values > 0:
            print(f"Negative value detected in the Weighting matrix. Total negative values: {negative_values}")
            
        wPndDF = pd.DataFrame(w)
        wPndDF.to_csv('wMatrix')
        
        # Compute the cholesky of w (banded). Then do the inverse. 
     
        ################################################################################################
        # Compute Covariance Matrix
        ################################################################################################    
        # Compute Covariance  
        # pdb.set_trace()
        
        if computeCovariance == True:
            covariance_A = np.linalg.inv(At.dot(W).dot(A))
        else:
            covariance_A = 'Covariance Not Computed'
    
    
    ##############################################################################################################
    # Step X: Perform Least Squares
    ##############################################################################################################
    
    
    if LeastSquaresMethod == 'Numpy':
        AW = np.transpose(trimmed_A).dot(W)
        Ve_xW = W.dot(Ve_x)
        coeff_list = np.linalg.lstsq(np.transpose(AW), Ve_xW, rcond=None)[0] # This has just been used for debugging to check if "Long" least squares leads to same results.
    elif LeastSquaresMethod == 'SciKit':
        testSKlearn = LinearRegression()
        testSKlearn.fit(trimmed_A, Ve_x, sample_weight=(np.diag(W)))
        coeff_list = testSKlearn.coef_
        coeff_list[0] = testSKlearn.intercept_
    elif LeastSquaresMethod == 'LongHand':
        coeff_list = np.linalg.inv(At.dot(W).dot(A)).dot(At).dot(W).dot(Ve_x[1:])
        
    else: 
        print("Did not select an applicable Least Squares Method")

    print_List = np.array(list(coeff_dict.keys()))
    
    n = 0
    for coef in print_List[trimmed_A_filt]:
        coeff_dict[coef] = coeff_list[n]
        n += 1
    
    #%% Save results to DataFrame
    coefficientDF = pd.DataFrame()
    
    coefficientDF = pd.concat((coefficientDF, pd.DataFrame.from_dict(AccelObj.AccelModelCoef, orient = 'index', columns= ['Accel Model'])))
    coefficientDF.loc['V_0'] = 0
    
    # Build Estimated Coefficient DF
    estimatedCoefficients = pd.DataFrame.from_dict(coeff_dict, orient = 'index', columns= ['Estimated Coefficients'])
    renameDict = {}
    for coeff in print_List:
        renameDict[coeff] = coeff[4:]
    estimatedCoefficients = estimatedCoefficients.rename(index = renameDict) 
    estimatedCoefficients.replace(0, np.nan, inplace=True)
    
    
    coefficientDF = pd.merge(coefficientDF,estimatedCoefficients,left_index=True, right_index=True)
    
    coefficientDF['Coefficient Estimate Error'] = coefficientDF['Accel Model'] - coefficientDF['Estimated Coefficients']
    
                
    #%% Compute Velocity Error Residuals
    V_error_model_terms = [coeff_dict['Est_V_0'], 
                           coeff_dict['Est_K_1']*Vx,  
                           coeff_dict['Est_K_0']*Ve_t, 
                           coeff_dict['Est_K_2']*intAx_2, 
                           coeff_dict['Est_K_3']*intAx_3,  
                           coeff_dict['Est_K_4']*intAx_4,  
                           coeff_dict['Est_K_5']*intAx_5]
    Error['V_error_model'] = sum(V_error_model_terms)*g 
    Error['Ve_x_Resid'] = Error['VelErr_x'] - Error['V_error_model']     

    #%% Save off results:
    if saveToPickel == True:
        Error.to_pickle(f"./ErrorDF_{N_model[0]}-{N_model[1]}.pkl")
        coefficientDF.to_pickle(f"./coefficientDF_{N_model[0]}-{N_model[1]}.pkl")
        
    return [coefficientDF, Error, covariance_A, A, Ve_x, W, LeastSquaresMethod]

### Set initial coefficients and parameters for script

Set the initial configuration parameters and logic that drives how results are computed. 

In [34]:
############################################################
#%% Initial Configuration Parameters
############################################################

# Set value for g

g = 9.791807  


# Set parameters for error added to Reference Position Vector

sigmaRPV = 0.001        # Standard deviation of Random noise centered at zero added to downtrack distance (meters)
tauRPV =  0            # Time Lag Error (seconds)
biasRPV = 0            # Bias error in RPV (meters) 

# Set number of Monte Carlo Runs

MCnum = 1

# Set custom coefficients for Accelerometer error model. Updates accelerometer coefficient error model 
# dictionary if ChangeDefaultCoeff is set to True.

CoeffDict = {'K_0': 5E-8}

# Used to determine how many coefficients to calculate.

N_model_start = 2     #  0 =  K_1 (Scale Factor), 1 = K_0 (Bias), 2 = K_2, etc. 
N_model_end = 5      #  0 = K_1 (Scale Factor), 1 = K_0 (Bias), 2 = K_2, etc.


# Clean up Model indicies and define Error Coefficient Names
N_model = [0,0]
# Fix indexing numbers
N_model[0] = N_model_start  ### REVIEW THIS
N_model[1]= N_model_end + 1

# Definition of corresponding coefficient names that will be computed based on above pararmeters
ModelList = ['K_1', 'K_0', 'K_2', 'K_3','K_4','K_5']


############################################################
#%% Initial Configuration Logic
############################################################

# If set to True, accelerometer model error will be updated with CoeffDict values set in intial parameters.

changeDefaultCoeff = True


# Generate New Trajectory. If set to True new Trajectory will be created and saved to .pkl file from EGI data.

generateNewTrajectory = False


# Generate New RPV. If set to True a new RPV will be generated and saved to .pkl file. If set to False, code will 
# to make sure and RPV with the parameters set in the intial configuration is available. If not availble a new RPV 
# be generated. 
generateNewRPV = True


# LeastSquaresMethod sets the method used for Least Squares Regression analaysis. Default is set to 'LongHand'
#  - 'LongHand':  Computes the least squares using numpy matrix multiplication. This is the only method that works for 
#                 Weighted Least Squares with correleated off diagonal values in the weighting matrix.
#  - 'Numpy':     Uses the least squares function from the numpy.linalg library. This method should not be used if using any sort of weighted least squares method.
#  - 'SciKit':    Computes the least squares regression using the SciKit library. This does not use any correlated off diagonal values. 

LeastSquaresMethod = 'LongHand'

# If set to True the least squares regression method for determining error coefficients will use a 
# Weighted Least Squares Method.

WLS = True

# If set to true the model will perform regression analysis for each term indiviually as well as the full model as defined above or look at each individual coefficient.

individualCoeffAnalysis = True

# Set start and stop time (seconds) to perform regression analysis on.
startStopTime = (0, -1)

## Run Monte Carlo Simulations


In [35]:
# Intialize Data Storage variables
coefficientEstimates = [];

# Run Monte Carlo Sims
for mc in range(MCnum):

    print(f'Running MC {mc}...', end="\r")  
    
    ########################t######################################################################################
    #%% Generate or import trajectory
    if generateNewTrajectory == True:      
        generateReferenceTrajectory()

    # Import Reference Trajectory
    referenceTrajectory = pd.read_pickle("./referenceTrajectory.pkl")


    ##############################################################################################################
    #%% Generate track reference position vectory

    # If generateNewRPV is set to False, check if an RPV exists with the specified parameters. If it does not
    # then set generateNewRPV to True so tha generateNewRPV runs anyways.
    if generateNewRPV == False:   
        generateNewRPV = not os.path.isfile(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")

    if generateNewRPV == True:    
        generateTrackRPV(referenceTrajectory, sigmaRPV, tauRPV, biasRPV)

    # Import trackRPV pickle file that matches configuration parameters
    trackRPV = pd.read_pickle(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")


    ##############################################################################################################
    #%% Generate Simulated Accelerometer for full model
    sensorSim, AccelObj = AccelSim(referenceTrajectory, N_model, changeDefaultCoeff, CoeffDict, g)
    
    
    #%% Perform Regression Analysis for full model
    if mc == 0:
        coefficientDF, Error, cov_A, A, Ve_x, W, LeastSquaresMethod = RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g, sigmaRPV, startStopTime, WLSoption = WLS, computeCovariance = True)
        coefficientCovariance = cov_A
    else:
        coefficientDF, Error, cov_A, A, Ve_x, W, LeastSquaresMethod = RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g, sigmaRPV, startStopTime, WLSoption = WLS, computeCovariance = False)
  
    coefficientEstimates.append(coefficientDF)
    
print('Completed MC runs')
    

Completed MC runs


# Monte Carlo Results Analysis

In [38]:
CoefficientEstimateErrors = pd.DataFrame()

K_1_Errors = []
K_0_Errors = []
K_2_Errors = []
K_3_Errors = []
K_4_Errors = []
K_5_Errors = []

for n in range(len(coefficientEstimates)):
    K_1_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_1'])
    K_0_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_0'])
    K_2_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_2'])
    K_3_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_3'])
    K_4_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_4'])
    K_5_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_5'])
    
CoefficientEstimateErrors['K_1_Errors'] = K_1_Errors
CoefficientEstimateErrors['K_0_Errors'] = K_0_Errors
CoefficientEstimateErrors['K_2_Errors'] = K_2_Errors
CoefficientEstimateErrors['K_3_Errors'] = K_3_Errors
CoefficientEstimateErrors['K_4_Errors'] = K_4_Errors
CoefficientEstimateErrors['K_5_Errors'] = K_5_Errors

Variances = np.diag(coefficientCovariance)

In [39]:
# If length of Coefficient Estimate Errors dataframe is 3 or more
if len(CoefficientEstimateErrors) >= 3:
    # print('K_1 Variance (real vs estimated):', CoefficientEstimateErrors['K_1_Errors'].std()**2, ' ', Variances[1])
    # print('K_0 Variance (real vs estimated):', CoefficientEstimateErrors['K_0_Errors'].std()**2, ' ', Variances[2])

    print('K_2 Variance (real vs estimated):', CoefficientEstimateErrors['K_2_Errors'].std()**2, ' ', Variances[1])
    print('K_3 Variance (real vs estimated):', CoefficientEstimateErrors['K_3_Errors'].std()**2, ' ', Variances[2])
    print('K_4 Variance (real vs estimated):', CoefficientEstimateErrors['K_4_Errors'].std()**2, ' ', Variances[3])
    print('K_5 Variance (real vs estimated):', CoefficientEstimateErrors['K_5_Errors'].std()**2, ' ', Variances[4])

K_2 Variance (real vs estimated): 2.726650201527623e-12   9.093680990971302e-10
K_3 Variance (real vs estimated): 1.1580929606381783e-13   4.237029726734616e-11
K_4 Variance (real vs estimated): 2.2181683818822318e-15   5.453566831763081e-13
K_5 Variance (real vs estimated): 3.864136235658828e-17   1.0768082779895156e-14


In [ ]:
print(np.sqrt(Variances))

[2.87516465e-06 2.97183095e-10 6.62082667e-12 2.50916053e-11
 2.88427774e-12 1.30930020e-14 4.40858238e-16]


In [40]:
print('K_1 Mean:', CoefficientEstimateErrors['K_1_Errors'].mean())
print('K_0 Mean:', CoefficientEstimateErrors['K_0_Errors'].mean())
print('K_2 Mean:', CoefficientEstimateErrors['K_2_Errors'].mean())
print('K_3 Mean:', CoefficientEstimateErrors['K_3_Errors'].mean())
print('K_4 Mean:', CoefficientEstimateErrors['K_4_Errors'].mean())
print('K_5 Mean:', CoefficientEstimateErrors['K_5_Errors'].mean())

K_1 Mean: nan
K_0 Mean: nan
K_2 Mean: 2.5409974483267055e-07
K_3 Mean: 8.047165105291427e-08
K_4 Mean: -8.461925036853243e-09
K_5 Mean: -1.3780192608680843e-09


In [41]:
print(coefficientDF)

      Accel Model  Estimated Coefficients  Coefficient Estimate Error
K_1  5.000000e-06                     NaN                         NaN
K_0  5.000000e-08                     NaN                         NaN
K_2  6.114000e-05            5.929199e-05                1.848014e-06
K_3  2.000000e-08           -4.262237e-07                4.462237e-07
K_4  6.000000e-09            6.212808e-08               -5.612808e-08
K_5  2.300000e-09            1.022777e-08               -7.927774e-09
V_0  0.000000e+00           -1.046255e-03                1.046255e-03


In [ ]:
      Accel Model  Estimated Coefficients  Coefficient Estimate Error
K_1  5.000000e-06                     NaN                         NaN
K_0  5.000000e-09                     NaN                         NaN
K_2  6.114000e-05            6.114068e-05               -6.785534e-10
K_3  2.000000e-08            1.966873e-08                3.312732e-10
K_4  6.000000e-09            5.995298e-09                4.702449e-12
K_5  2.300000e-09            2.304385e-09               -4.385152e-12
V_0  0.000000e+00           -4.434971e-07                4.434971e-07